In this notebook we go through the adaptive learning process used for the paper. We provide several different approaches: "IFT", "IL", "AL", "SML", "EWC", "GEM", "ALG", "eALG", "GIL", "Scratch". For detailed explenations of the methods, we refer to the paper.

This config can be modified to setup the experiment.

In [ ]:
adaptive_learning_config = {
    "data": {
        "folder": "data/target_data"
    },
    "training": {
        "epochs": 30
    },
    "adaptive_learning": {
        "number_of_years": 5.1,
        "output_file_name": "test"
    }
}

We can use this functionality to do all Adaptive Learning experiments at once. The results will be saved in a output folder. This folder contains the performance for each update interval and method. Also, a respective plot is generated.

In [ ]:
# Import general dependencies
from copy import deepcopy

# Import project-specific dependencies
from src.utils.adaptive_learning.run_adaptive_learning import run_adaptive_learning, update_config
from src.utils.utilfunctions import read_config
from src.plots.plot_over_update import plot_aggregated_errors_from_csv
import json

# Define setup
update_periods = [1, 2, 3]         # update period in months
modes = ["IL"]                 # Options: "IFT", "IL", "AL", "SML", "EWC", "GEM", "ALG", "eALG", "GIL", "Scratch"
experiment_config = adaptive_learning_config

# base config
base_config_path = "models/configs/al_base_config.json"
base_config = read_config(base_config_path)

name = experiment_config["adaptive_learning"]["output_file_name"]

for k in range(len(modes)):
    
    experiment_config["adaptive_learning"]["mode"] = modes[k]
    experiment_name = name + f"_{modes[k]}"

    for i in range(len(update_periods)):

        # override the base config with the experiment config
        config_ = deepcopy(base_config)
        experiment_config["adaptive_learning"]["update_interval"] = update_periods[i] * 2880
        experiment_config["adaptive_learning"]["train_val_interval"] = update_periods[i] * 2880
        experiment_config["adaptive_learning"]["test_interval"] = update_periods[i] * 2880

        experiment_config["adaptive_learning"]["output_file_name"] = experiment_name + f"update_{update_periods[i]}_month"
        if "adaptive_learning" not in config_:
            config_["adaptive_learning"] = {}
        config_ = update_config(config_, experiment_config)
        try:
            # run the continuous learning with i_th mode
            output_path, update_steps = run_adaptive_learning(config_)
            
        except Exception as e:
            print(f"Skipping mode '{modes[k]}' with update_period '{update_periods[i]}' due to error: {e}")
            continue

        # Save config and make plots
        with open(f"{output_path}/config.json", "w") as f:
            json.dump(experiment_config, f, indent=4)
        
        plot_aggregated_errors_from_csv(output_path + "/errors.csv", 'rmse', 
                                        num_update_steps=(update_steps+1), 
                                        output_path=output_path, 
                                        update_period=update_periods[i])
